In [1]:
from api_neurotask import *
from baselines.metrics import get_R2
from baselines.forecast import *
import pynapple as nap
import nemos as nmo

In [2]:
data = nap.load_file("001056/sub-Animal-1-&-2/sub-Animal-1-&-2.nwb")
df , bin = get_dataframe(data,filter_result=[b'R'])

Data loaded with bin size of 10.0 ms


In [3]:
new_bin_size = 20
df10 = rebin(df,prev_bin_size = bin ,new_bin_size = new_bin_size)

In [4]:
df10.head()

,session,trial_id,Neuron0,Neuron1,Neuron2,Neuron3,Neuron4,Neuron5,Neuron6,Neuron7,...,result,EventTarget_Onset,EventGo_cue,EventBump,target_dir,hand_vel_y,hand_vel_x,hand_pos_y,hand_pos_x,bump_dir
0,1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,b'R',False,False,False,0.0,13.844993,-9.663045,-40.768042,-2.777333,1.570796
1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,b'R',False,False,False,0.0,12.197587,-8.697542,-40.507926,-2.961398,1.570796
2,1,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,b'R',False,False,False,0.0,10.629333,-7.434564,-40.279349,-3.122960,1.570796
3,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,b'R',False,False,False,0.0,9.039913,-6.239253,-40.083160,-3.259433,1.570796
4,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,b'R',False,False,False,0.0,7.627443,-5.119604,-39.916148,-3.373000,1.570796


In [5]:
X_train_list, y_train_list, X_val_list, y_val_list, X_test_list, y_test_list= process_data_forecast_mimo(df10, bins_before=5,bins_after=1,
                                                 training_range=[0, 0.7], valid_range=[0.7, 0.8], testing_range=[0.8, 1], 
                                        filter_g=True, bin = new_bin_size, sigma= 50,scale = True)

In [6]:
print("There are", sum(X.shape[2] for X in X_train_list), "unique neurons in the dataset in",len(X_train_list),"different sessions")

There are 461 unique neurons in the dataset in 4 different sessions


In [7]:
_, _, _, _, _, y_test_true = process_data_forecast_mimo(df10, bins_before=5,bins_after=20,
                                                    training_range=[0, 0.7], valid_range=[0.7, 0.8], testing_range=[0.8, 1], 
                                            filter_g=True, bin = new_bin_size, sigma= 50,scale=True)

In [8]:
def glm_forecast(X_train_list, X_test_list,y_train_list, y_test_list,y_test_true, regularizer_strength=0.15, solver = "LBFGS",n_steps=20,clip = True  ):

    for i in range(len(X_train_list)):
        X_flat_train=X_train_list[i].reshape(X_train_list[i].shape[0],(X_train_list[i].shape[1]*X_train_list[i].shape[2]))

        X_flat_test = X_test_list[i].reshape(X_test_list[i].shape[0],(X_test_list[i].shape[1]*X_test_list[i].shape[2]))

        y_flat_train=y_train_list[i].reshape(y_train_list[i].shape[0],(y_train_list[i].shape[1]*y_train_list[i].shape[2]))

        y_flat_test = y_test_list[i].reshape(y_test_list[i].shape[0],(y_test_list[i].shape[1]*y_test_list[i].shape[2]))

        #Fit model|
        model = nmo.glm.PopulationGLM(
            regularizer=nmo.regularizer.Ridge(regularizer_strength=regularizer_strength, solver_name=solver),
            ).fit(X_flat_train, y_flat_train)


        # Recursive prediction
        
        initial_input = X_flat_test
        predictions = []
        neurons = X_train_list[i].shape[2]
        #clip_range=(-1, 1)

        for _ in range(n_steps):
            prediction = model.predict(initial_input)
            if clip: prediction = np.clip(prediction, -2,2)
            predictions.append(prediction)
            # Update the input with the prediction for the next recursive step
            
            initial_input = np.concatenate((initial_input[:,neurons:],prediction),axis=1)

        pred = np.concatenate(predictions, axis=1)

        #y_testtrue = y_test_true[i].reshape(y_test_true[i].shape[0],(y_test_true[i].shape[1]*y_test_true[i].shape[2]))
        r2s = get_R2(y_test_true[i][:-n_steps,:,:],pred[:-n_steps,:].reshape(y_test_true[i][:-n_steps,:,:].shape))

        print('session', i+1,'R2s:', r2s, '5: ', r2s[:5].mean(), '20: ', r2s.mean())


        mse = (np.square(y_test_true[i][:-n_steps,:,:] -  pred[:-n_steps,:].reshape(y_test_true[i][:-n_steps,:,:].shape))).mean(axis=(0,2))
        print('session', i+1,'MSE:', mse, '5: ', mse[:5].mean(), '20: ', mse.mean())
        mae = np.absolute((y_test_true[i][:-n_steps,:,:] -  pred[:-n_steps,:].reshape(y_test_true[i][:-n_steps,:,:].shape))).mean(axis=(0,2))
        print('session', i+1,'MAE:', mae, '5: ', mae[:5].mean(), '20: ', mae.mean())

In [9]:
glm_forecast(X_train_list, X_test_list,y_train_list, y_test_list,y_test_true, regularizer_strength=0.15, solver = "LBFGS",n_steps=20,clip = True  )

session 1 R2s: [0.58540649 0.48761498 0.38981378 0.30389952 0.2360876  0.18700758
 0.15869809 0.13844853 0.12410127 0.1139107  0.10660697 0.10131277
 0.09740296 0.09450929 0.0923609  0.0907608  0.08956725 0.08867871
 0.0880222  0.08754316] 5:  0.4005644750093584 20:  0.18308767688042615
session 1 MSE: [0.00924546 0.01142623 0.01360721 0.01552312 0.01703532 0.01812977
 0.01876106 0.01921268 0.01953269 0.01975988 0.01992243 0.02004
 0.02012681 0.02019141 0.02023999 0.02027684 0.02030481 0.02032578
 0.02034108 0.02035178] 5:  0.013367471139487452 20:  0.018217717914364155
session 1 MAE: [0.07465244 0.08246972 0.08951232 0.0951927  0.09938461 0.10226464
 0.10382113 0.10489719 0.10563383 0.10614035 0.10649361 0.10674289
 0.10692484 0.10705949 0.10715967 0.10723451 0.10728988 0.10733019
 0.10735904 0.10737901] 5:  0.08824235837457384 20:  0.10174710279020376
session 2 R2s: [0.54665042 0.45822642 0.36829262 0.28572148 0.21531531 0.15782304
 0.12784933 0.10543548 0.08889952 0.07674553 0.067800